# Configuration model

In [1]:
import networkx as nx
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from itertools import chain 
from itertools import zip_longest

## Parameters

The algorithm has 2 input parameters: 
* N: number of nodes
* p_k : nodes degree distribution

Insert the parameters of the algorithm (need to add the checks!!!!!!!!!)

In [2]:
# Insert Number of Nodes
N = int(input("\nInsert number of nodes:\n"))


Insert number of nodes:
10000


### Distribution choice
Try at least with a Poisson (---> generates a ER network) and with a powerlaw distribution (--> generates a SF network)

#### Powerlaw distribution

In [3]:
# distribution exponent
gamma = 2.2

seq=nx.utils.powerlaw_sequence(N,exponent=gamma)
# round to integer values in the range [0,N]
sequence=[min(N, max( int(round(s)),0 )) for s in seq]

# The degree sequence needs to have an even sum
while sum(sequence) % 2 != 0:
    seq=nx.utils.powerlaw_sequence(N,exponent=gamma)
    sequence=[min(N, max( int(round(s)),0 )) for s in seq]

sequence

[9,
 1,
 4,
 7,
 1,
 2,
 5,
 1,
 6,
 6,
 3,
 1,
 103,
 2,
 8,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 1,
 1,
 9,
 2,
 1,
 2,
 1,
 3,
 1,
 2,
 1,
 1,
 14,
 1,
 1,
 14,
 7,
 4,
 3,
 5,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 5,
 1,
 2,
 1,
 4,
 4,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 5,
 7,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 1,
 2,
 1,
 6,
 1,
 6,
 65,
 2,
 1,
 13,
 3,
 3,
 2,
 4,
 2,
 2,
 2,
 2,
 1,
 7,
 3,
 6,
 1,
 1,
 1,
 1,
 3,
 1,
 4,
 1,
 2,
 2,
 2,
 3,
 4,
 1,
 1,
 1,
 4,
 15,
 1,
 2,
 2,
 15,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 3,
 2,
 1,
 1,
 15,
 2,
 1,
 2,
 2,
 11,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 3,
 3,
 1,
 1,
 1,
 4,
 4,
 2,
 1,
 1,
 3,
 1,
 10,
 1,
 9,
 2,
 3,
 1,
 5,
 2,
 9,
 1,
 3,
 2,
 3,
 2,
 2,
 1,
 2,
 2,
 3,
 1,
 4,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 2,
 1,
 3,
 1,
 1,
 20,
 1,
 1,
 2,
 2,
 5,
 1,
 5,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 2,
 6,
 1,
 1,
 2,
 1,
 2,
 10,
 1,
 3,
 2,
 1,
 2,
 2,
 2,
 9,
 1,
 1,
 2,
 2,
 3,
 3,
 10,
 2,
 5,
 2,
 1,
 1,
 1,
 7

#### Poisson distribution

## Algorithm
We need now to build a random graph/network with this degree sequence. <br>
Let's build before an empty graph

In [4]:
G = nx.empty_graph(N, create_using = nx.MultiGraph) # MultiGraph because of self-loops

We need now to create the vector of slots, i.e. a list of available degree-repeated nodes. <br>

Then, we'll shuffle the slots list in order to get a random sequence of node pairs.

In [5]:
slots = list(chain.from_iterable([n] * d for n, d in enumerate(sequence)))
slots

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 20,
 21,
 22,
 22,
 22,
 23,
 23,
 23,
 24,
 25,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 28,
 29,
 29,
 30,
 31,
 31,
 31,
 32,
 33,
 33,
 34,
 35,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 3

Shuffle the slots list

In [6]:
np.random.shuffle(slots)
slots

[4433,
 7620,
 2548,
 7967,
 2434,
 6287,
 3412,
 2488,
 2055,
 9695,
 3483,
 1671,
 5018,
 2716,
 7900,
 2908,
 7967,
 5975,
 1640,
 7967,
 6175,
 6746,
 6578,
 5503,
 1029,
 5018,
 4707,
 6854,
 7594,
 7743,
 7649,
 4726,
 6310,
 7620,
 1718,
 2055,
 2446,
 277,
 1693,
 369,
 7594,
 4322,
 1737,
 9787,
 5085,
 7967,
 157,
 9824,
 8102,
 5785,
 5709,
 8232,
 737,
 3916,
 4156,
 6527,
 981,
 3032,
 7620,
 8749,
 7923,
 777,
 825,
 8727,
 3083,
 5012,
 5052,
 1427,
 8457,
 2612,
 7358,
 8110,
 7743,
 418,
 4694,
 2176,
 3747,
 3732,
 3661,
 7967,
 3483,
 9270,
 3628,
 9976,
 7743,
 8351,
 9993,
 7967,
 2176,
 7967,
 7967,
 7743,
 7933,
 4295,
 7967,
 8202,
 4952,
 4924,
 7830,
 6517,
 7221,
 1502,
 7743,
 5018,
 5352,
 2208,
 7967,
 1577,
 7620,
 7967,
 1665,
 6933,
 8749,
 6802,
 7620,
 623,
 4131,
 1737,
 277,
 7620,
 2767,
 5657,
 6972,
 3863,
 7743,
 7967,
 2171,
 2378,
 1767,
 8536,
 7967,
 7743,
 8310,
 7967,
 4693,
 2935,
 9882,
 3867,
 7743,
 6585,
 7967,
 181,
 8049,
 5278,
 79

We can now bipartition the shuffled slot vector to have a random pairing of nodes. <br>
In that way we can use the add_edges_from function of networkx

In [7]:
half = len(slots) // 2

out_slots, in_slots = slots[:half], slots[half:]
G.add_edges_from(zip(out_slots, in_slots)) # with zip we create a series of tuples from the 2 lists

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Let's save the graph

In [8]:
nx.write_pajek(G, "test.net")
#nx.write_gexf(G, "test.gexf")

## Network descriptors

In [9]:
print("Number of nodes: {}".format(G.number_of_nodes()))
#G.nodes()

print("Number of edges: {}".format(G.number_of_edges()))
#G.edges()

degrees_values = [val for key,val in G.degree()] # degree for each node
#degrees_values

print("Min degree: {}".format(np.min(degrees_values)))
print("Max degree: {}".format(np.max(degrees_values)))
print("Avg degree: {}".format(round(np.mean(degrees_values), 4)))
G = nx.Graph(G)

print("Clustering: {}".format(round(nx.average_clustering(G), 4)))
print("Assortativity: {}".format(round(nx.degree_assortativity_coefficient(G), 4)))
if nx.is_connected(G):
    print("Avg path length: {}".format(round(nx.average_shortest_path_length(G), 4)))
    print("Diameter: {}".format(nx.diameter(G)))

Number of nodes: 10000
Number of edges: 27345
Min degree: 1
Max degree: 7016
Avg degree: 5.469
Clustering: 0.129
Assortativity: -0.1235


## Plots
Maybe plotting with Gephi, .....

### Plotting the network

In [10]:
#nx.draw(G, alpha = .3, edge_color = '#40a6d1', node_color = '#40a6d1', node_size=30, with_labels = True)

### Plotting the distribution

K_distr() function  for visualizing the distribution using linear or log-log scale.

Degree distribution of Barabási–Albert network is k**(-3) and so it gives a straight line in log-log scale.

In [11]:
# Plotting

def k_distrib(graph, scale='lin', colour='#40a6d1', alpha=.8, fit_line=False, expct_lo=1, expct_hi=10, expct_const=1):
    
    plt.close()
    num_nodes = graph.number_of_nodes()
    max_degree = 0
    
    # Calculate the maximum degree to know the range of x-axis
    for n in graph.nodes():
        if graph.degree(n) > max_degree:
            max_degree = graph.degree(n)
    
    # X-axis and y-axis values
    x = []
    y_tmp = []
    
    # Loop over all degrees until the maximum to compute the portion of nodes for that degree
    for i in range(max_degree + 1):
        x.append(i)
        y_tmp.append(0)
        for n in graph.nodes():
            if graph.degree(n) == i:
                y_tmp[i] += 1
        y = [i / num_nodes for i in y_tmp] 
    
    # Check for the lin / log parameter and set axes scale
    if scale == 'log':
        plt.xscale('log')
        plt.yscale('log')
        plt.title('Degree distribution (log-log scale)')
        plt.ylabel('log(P(k))')
        plt.xlabel('log(k)')
        plt.plot(x, y, linewidth = 0, marker = 'o', markersize = 8, color = colour, alpha = alpha)
        
        if fit_line:
            # Add theoretical distribution line k^-3
            # Note that you need to parametrize it manually
            w = [a for a in range(expct_lo,expct_hi)]
            z = []
            for i in w:
                x = (i**-3) * expct_const # set line's length and fit intercept
                z.append(x)

            plt.plot(w, z, 'k-', color='#7f7f7f')
            
    else:
        plt.plot(x, y, linewidth = 0, marker = 'o', markersize = 8, color = colour, alpha = alpha)
        plt.title('Degree distribution (linear scale)')
        plt.ylabel('P(k)')
        plt.xlabel('k')

    plt.show()


* Linear scale example

In [ ]:
k_distrib(graph=G, colour='#40a6d1', alpha=.8)

* Log-log scale example

In [ ]:
k_distrib(graph=G, colour='#40a6d1', scale='log', alpha=.8, fit_line=True, expct_lo=1, expct_hi=40, expct_const=35)

### Degree distribution plots

In [ ]:
# Auxiliary function to create logaritmically spaced bins (for log-log histogram) by specifying the number of bins
def create_log_bins(degrees, num = 20):
    bins = np.logspace(np.log10(np.min(degrees)), np.log10(np.max(degrees)), num)
    bins = np.array(bins)
    return bins

In [ ]:
# PDF histogram in linear scale
def plot_linear_PDF(G, name='', nbins = 15):
    degrees = [G.degree(n) for n in G.nodes()]
    #plt.figure(figsize=(12,8))
    plt.title('PDF in linear scale', fontsize=15)
    plt.xlabel('Degree', fontsize=13)
    plt.ylabel('PDF', fontsize=13)
    plt.hist(degrees, bins=nbins, density = True, cumulative = False)
    plt.tight_layout()
    plt.style.use('ggplot')
        
#plot_linear_PDF(G)

In [ ]:
# PDF histogram in Log-Log scale
def plot_loglog_PDF(G, name="", nbins=20):
    degrees = [G.degree(n) for n in G.nodes()]
    
    # creating logaritmically spaced bins
    bins = create_log_bins(degrees, num = nbins)
    
    #plt.figure(figsize=(12,8))
    plt.title('PDF in log-log scale',  fontsize=15)
    plt.xlabel('Degree', fontsize=13)
    plt.ylabel('PDF', fontsize=13)
    plt.yscale('log')
    plt.xscale('log')
    plt.hist(degrees, bins=bins, density = True, cumulative = False)
    plt.tight_layout()
    #plt.style.use('ggplot')

#plot_loglog_PDF(G)

In [ ]:
# CCDF histogram in linear scale
def plot_linear_CCDF(G, name="", nbins=30):
    degrees = [G.degree(n) for n in G.nodes()]
    #plt.figure(figsize=(12,8))
    plt.title('CCDF in linear scale', fontsize=15)
    plt.xlabel('Degree', fontsize=13)
    plt.ylabel('CCDF', fontsize=13)
    plt.hist(degrees, bins=nbins, density = True, cumulative = -1)
    plt.tight_layout()
    #plt.style.use('ggplot')

#plot_linear_CCDF(G)

In [ ]:
# CCDF histogram in Log-Log scale
def plot_loglog_CCDF(G, name="", nbins=30):
    degrees = [G.degree(n) for n in G.nodes()]
    
    # creating logaritmically spaced bins
    bins = create_log_bins(degrees, num=nbins)
    
    #plt.figure(figsize=(12,8))
    plt.title('CCDF in log-log scale', fontsize=15)
    plt.xlabel('Degree', fontsize=13)
    plt.ylabel('CCDF', fontsize=13)
    plt.yscale('log')
    plt.xscale('log')
    plt.hist(degrees, bins=bins, density = True, cumulative = -1)
    plt.tight_layout()
    #plt.style.use('ggplot')
    

#plot_loglog_CCDF(G)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plot_linear_PDF(G, nbins=20) 
plt.subplot(1, 2, 2)
plot_loglog_PDF(G, nbins=20)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plot_linear_CCDF(G, nbins=20) 
plt.subplot(1, 2, 2)
plot_loglog_CCDF(G, nbins=20)
plt.show()

## Estimation of the exponent for the empirical degree distributions of BA and CM(SF), for the networks of size N>=1000.

We can use the powerlaw Python package described here: https://arxiv.org/abs/1305.0215, which is based on a MLE

In [ ]:
#pip install powerlaw
#pip install mpmath 

In [ ]:
import powerlaw
degrees = [G.degree(n) for n in G.nodes()]
#results = powerlaw.Fit(degrees, discrete = True)
results = powerlaw.Fit(degrees)
#print(round(results.power_law.alpha, 4))
results.truncated_power_law.parameter1 # power law  parameter (alpha)
#results.truncated_power_law.parameter2 # exponential cut-off parameter (beta)
#print(results.power_law.xmin)

### Using Linear Regression